In [1]:
# ビット反転問題
#
# 評価関数を |yt - y| >= 0
#
# Goal:
# x0 = 1 -> y0 = 0
# x0 = 0 -> y0 = 1

# 評価関数の分析
# （間違っている時に１になる（期待値と差がある））
#
# x0=1 で w0,0x + b > 0の時
# |d| = |0 - f(w0,0x + b)| = |0 - 1| = 1
# x0=1 で w0,0x + b <= 0の時
# |d| = |0 - f(w0,0x + b)| = |0 - 0| = 0
# x0=0 で w0,0x + b > 0の時
# |d| = |1 - f(w0,0x + b)| = |1 - 1| = 0
# x0=0 で w0,0x + b <= 0の時
# |d| = |1 - f(w0,0x + b)| = |1 - 0| = 1
#
# 上記の式で、絶対値を外すと差の方向（符号）も見れる
#
# 学習アルゴリズムの設計と実装の発表
# 1. 与えられたデータxに対して出力y0を計算
# 2. 正解 -y0 > 0の時 w0,0 = w0,0 + αd
# 3. 正解 -y0 < 0の時 w0,0 = w0,0 - αd
# 4. 1へ戻る
#
# グラフw0,0, x=0, x=1を示す
# 初期値 w0,0=0.3, b=0.1

class Neuron:
    def __init__(self, X, Y, w00, b, alpha):
        self._X = X
        self._Y = Y
        self._w = w00
        self._b = b
        self._alpha = alpha
        
    def learn(self, loop_num):
        log = []
        for num in range(loop_num):
            success = 0
            for i in range(len(X)):
                val, res = self.eval_func(self._X[i], self._Y[i])
                if res:
                    success += 1
                else:
                    if val < 0:
                        self._w -= self._alpha
                    elif val > 0:
                        self._w += self._alpha
            if (num+1) % 10 == 0:
                print("[{}] _w = {}".format(num+1, self._w))
            if success == len(self._X):
                break
        return (self._w, self._b, num+1)
    
    def eval_func(self, x, y):
        y0 = self._w * x + self._b
        return y-y0, y-y0 > 0


In [2]:
X = [0, 1]
Y = [1, 0]
w00 = 0.3
b = 0.1
alpha = 0.01
n = Neuron(X, Y, w00, b, alpha)
(w_, b_, num) = n.learn(1000)
print('result: [{}] _w = {}, _b = {}'.format(num, w_, b_))

[10] _w = 0.1999999999999999
[20] _w = 0.09999999999999985
[30] _w = -1.4224732503009818e-16
[40] _w = -0.10000000000000013
result: [41] _w = -0.10000000000000013, _b = 0.1


In [3]:
# numpy版
import numpy as np

class Perceptron:
    def __init__(self, w, b, f, alpha):
        self.w = np.concatenate((w, b), axis=1)
        self.f = f
        self.alpha = np.array([alpha, 0]) # 後でwと演算時に使用、bの増減値を0
        
    def output(self, x):
        x = np.concatenate((x, [[1,],]))  # バイアス用の係数1を追加
        return self.f(self.w.dot(x))
    
    def learn(self, X, Yt, loop_cnt):
        # 学習回数だけループ
        for num in range(loop_cnt):
            # y = f(wx+b)
            y = np.array([self.output(x[:, np.newaxis]) for x in X])
            # y_ = Yt - f(wx+b)
            y_ = Yt - y
            
            # 判定
            if len(np.where(y_==0)[0]) == len(y_):
                # すべて0ならば終了
                break
            elif len(np.where(y_>0)[0]) > len(np.where(y<=0)[0]):
                # 1が多ければalphaを加算
                self.w += self.alpha
            else:
                # -1が多ければalphaを減算
                self.w -= self.alpha
        return num+1, self.w[0,0], self.w[0,1]

def f(u):
    return 1 if u > 0 else 0
# 
w00 = np.array([[0.3]])
b = np.array([[0.1]])
alpha = 0.01
p0 = Perceptron(w00, b, f, alpha)

# 入力
X = np.array(([0, 1]))[:, np.newaxis]
# 期待値
Yt = np.array(([1, 0]))
# 学習回数
loop_cnt = 100
# 実行
(num, w00_, b_) = p0.learn(X, Yt, loop_cnt)
print('start : w00={}, b={}'.format(w00[0,0], b[0,0]))
print('result: w00={}, b={}, num={}'.format(w00_, b_, num))

start : w00=0.3, b=0.1
result: w00=-0.10000000000000013, b=0.1, num=41
